<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


## Step 1 : Load the data

In [2]:
import pandas as pd
import numpy as np
df= pd.read_csv('https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')

In [3]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## Step 2 :Clean the data

In [4]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [5]:
df.describe(exclude='number').T

,count,unique,top,freq
customerID,7043,7043,2725-IWWBA,1
gender,7043,2,Male,3555
Partner,7043,2,No,3641
Dependents,7043,2,No,4933
PhoneService,7043,2,Yes,6361
MultipleLines,7043,3,No,3390
InternetService,7043,3,Fiber optic,3096
OnlineSecurity,7043,3,No,3498
OnlineBackup,7043,3,No,3088
DeviceProtection,7043,3,No,3095


In [6]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [7]:
df['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [8]:
X = df.drop(['Churn'],axis = 1)
y = df['Churn']

In [9]:
df.shape,y.shape,X.shape

((7043, 21), (7043,), (7043, 20))

In [10]:
y = y.replace({'Yes':1, 'No':0})
y.value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

In [11]:
y

0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: Churn, Length: 7043, dtype: int64

In [12]:
y = np.array(y.values)

In [13]:
y

array([0, 0, 1, ..., 0, 1, 0])

## Step 3 : Create and fit a baseline Keras MLP model to the data

In [14]:
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
#categories = X.describe(exclude='number').columns
enc = OrdinalEncoder()
scaler = MinMaxScaler()

X = enc.fit_transform(X)

In [15]:
X

array([[5.375e+03, 0.000e+00, 0.000e+00, ..., 2.000e+00, 1.420e+02,
        2.505e+03],
       [3.962e+03, 1.000e+00, 0.000e+00, ..., 3.000e+00, 4.980e+02,
        1.466e+03],
       [2.564e+03, 1.000e+00, 0.000e+00, ..., 3.000e+00, 4.360e+02,
        1.570e+02],
       ...,
       [3.367e+03, 0.000e+00, 0.000e+00, ..., 2.000e+00, 1.370e+02,
        2.994e+03],
       [5.934e+03, 1.000e+00, 1.000e+00, ..., 3.000e+00, 7.950e+02,
        2.660e+03],
       [2.226e+03, 1.000e+00, 0.000e+00, ..., 0.000e+00, 1.388e+03,
        5.407e+03]])

In [16]:
X = scaler.fit_transform(X)

In [17]:
X

array([[0.76327748, 0.        , 0.        , ..., 0.66666667, 0.08964646,
        0.38361409],
       [0.56262425, 1.        , 0.        , ..., 1.        , 0.31439394,
        0.2245023 ],
       [0.36410111, 1.        , 0.        , ..., 1.        , 0.27525253,
        0.02404288],
       ...,
       [0.47813121, 0.        , 0.        , ..., 0.66666667, 0.0864899 ,
        0.45849923],
       [0.84265834, 1.        , 1.        , ..., 1.        , 0.50189394,
        0.40735069],
       [0.31610338, 1.        , 0.        , ..., 0.        , 0.87626263,
        0.8280245 ]])

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5634, 20), (5634,), (1409, 20), (1409,))

In [19]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(20, input_dim=20, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(12, activation='sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

In [29]:
#from sklearn.model_selection import StratifiedKFold
#from sklearn.model_selection import cross_val_score

# evaluate using 10-fold cross validation
#kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
#results = cross_val_score(model, X, y, cv=kfold)
#print(results.mean())

0.7346301674842834


In [31]:
model.fit(X_train, y_train, 
          validation_data=(X_test,y_test), 
          epochs=100, 
          batch_size=10
         )

In [33]:
print(f"Accuracy: {accuracy_score(np.round(model.predict(X_test)),y_test)}")

Accuracy: 0.8062455642299503


## Step 4 : Hyperparameter tune

### Tune Batch Size and Number of Epochs

In [61]:
# grid search epochs, batch size and optimizer
#optimizers = ['sgd', 'rmsprop', 'adam', 'adagrad']
init = ['glorot_uniform', 'normal', 'uniform']
epochs = np.array([20, 40, 60,100])
batches = np.array([5,10, 20, 40, 60, 80])
learning_rate = np.array([0.0001, 0.001, 0.01, 0.1])
dropout_rate = np.array([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

param_grid = dict(batch_size=batches, epochs=epochs)

#dict(optimizer=optimizers, nb_epoch=epochs, dropout_rate=dropout_rate, batch_size=batches, init=init) 

grid = GridSearchCV(estimator=model, param_grid=param_grid,n_jobs=-1,cv = 3)

grid_result = grid.fit(X, y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


Best: 0.803777 using {'batch_size': 80, 'epochs': 40}


In [62]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Means: 0.7980973930008859, Stdev: 0.0025887297083466067 with: {'batch_size': 5, 'epochs': 20}
Means: 0.8022149697075204, Stdev: 0.0033478987470162407 with: {'batch_size': 5, 'epochs': 40}
Means: 0.7995172465167465, Stdev: 0.0055082298846878385 with: {'batch_size': 5, 'epochs': 60}
Means: 0.7965355812360766, Stdev: 0.007460641642388708 with: {'batch_size': 5, 'epochs': 100}
Means: 0.7998012204019925, Stdev: 0.0009469804276526687 with: {'batch_size': 10, 'epochs': 20}
Means: 0.8012210756866123, Stdev: 0.0028086545858455082 with: {'batch_size': 10, 'epochs': 40}
Means: 0.8036348188057146, Stdev: 0.003880290531230437 with: {'batch_size': 10, 'epochs': 60}
Means: 0.7975294553859488, Stdev: 0.0048523691777533975 with: {'batch_size': 10, 'epochs': 100}
Means: 0.8016470219370284, Stdev: 0.002248739934784251 with: {'batch_size': 20, 'epochs': 20}
Means: 0.7976714410041182, Stdev: 0.002693945410567787 with: {'batch_size': 20, 'epochs': 40}
Means: 0.7992332691024451, Stdev: 0.0019389583794042476 

### Tune the Training Optimization Algorithm

In [75]:
def create_model1(optimizer = 'adam'):
    # create model
    model = Sequential()
    model.add(Dense(20, input_dim=20, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(12, activation='sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model1, epochs=40, batch_size=80, verbose=0)

# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'adagrad', 'adadelta', 'adam', 'adamax', 'nadam']

param_grid = dict(optimizer=optimizer)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

grid_result = grid.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


Best: 0.803493 using {'optimizer': 'nadam'}


In [76]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.734630 (0.004601) with: {'optimizer': 'SGD'}
0.798807 (0.002169) with: {'optimizer': 'RMSprop'}
0.734630 (0.004601) with: {'optimizer': 'adagrad'}
0.569076 (0.217555) with: {'optimizer': 'adadelta'}
0.801647 (0.001949) with: {'optimizer': 'adam'}
0.795684 (0.004930) with: {'optimizer': 'adamax'}
0.803493 (0.001024) with: {'optimizer': 'nadam'}


### Tune Learning Rate

In [113]:
from tensorflow.keras.optimizers import Nadam

def create_model(learn_rate=0.01):
 # create model
    model = Sequential()
    model.add(Dense(20, input_dim=20, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(12, activation='sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
# Compile model
    optimizer = Nadam(lr=learn_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, epochs=40, batch_size=80, verbose=0)

# define the grid search parameters
learn_rate = [0.0001, 0.001, 0.01, 0.1,0.2,0.3]


param_grid = dict(learn_rate=learn_rate)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

grid_result = grid.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.802783 using {'learn_rate': 0.01}


### Tune Network Weight Initialization

In [115]:
def create_model(init_mode='uniform'):
 # create model
    model = Sequential()
    model.add(Dense(20, input_dim=20, kernel_initializer=init_mode,activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(12,  kernel_initializer=init_mode,activation='sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(1,  kernel_initializer=init_mode,activation='sigmoid'))
# Compile model
   
    model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, epochs=40, batch_size=80, verbose=0)

# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

grid_result = grid.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.802783 using {'init_mode': 'glorot_normal'}


### Tune the Neuron Activation Function

In [116]:
def create_model(activation='relu'):
 # create model
    model = Sequential()
    model.add(Dense(20, input_dim=20, kernel_initializer='glorot_normal',activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(12,  kernel_initializer='glorot_normal',activation='sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(1,  kernel_initializer='glorot_normal',activation='sigmoid'))
# Compile model
   
    model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, epochs=40, batch_size=80, verbose=0)

# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

grid_result = grid.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.803919 using {'activation': 'relu'}


### Tune the Number of Neurons in the Hidden Layer

In [117]:
def create_model(neurons=1):
 # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=20, kernel_initializer='glorot_normal',activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(neurons,  kernel_initializer='glorot_normal',activation='sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(1,  kernel_initializer='glorot_normal',activation='sigmoid'))
# Compile model
   
    model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, epochs=40, batch_size=80, verbose=0)

# define the grid search parameters
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(neurons=neurons)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

grid_result = grid.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.802499 using {'neurons': 30}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?